In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

In [3]:
# Load data
train_data = pd.read_csv('data_processed/train.csv')
val_data = pd.read_csv('data_processed/val.csv')

train_data

,Date,HomeTeam,AwayTeam,Result,Hodds,Dodds,Aodds,home_wins,home_draws,home_losses,...,home_oppos_yellowcards,home_oppos_redcards,away_fouls,away_yellowcards,away_redcards,away_cornerkicks,away_oppos_cornerkicks,away_oppos_fouls,away_oppos_yellowcards,away_oppos_redcards
0,2010-12-26,Aston Villa,Tottenham,A,3.042857,3.285714,2.341429,0.2,0.4,0.4,...,1.000000,0.000000,10.2,1.2,0.0,5.6,10.333333,10.000000,0.666667,0.000000
1,2010-12-26,Blackburn,Stoke,A,2.240000,3.214286,3.285714,0.2,0.2,0.6,...,2.333333,0.333333,9.4,0.8,0.0,5.8,7.333333,10.666667,2.000000,0.000000
2,2010-12-26,Fulham,West Ham,A,1.932857,3.328571,4.092857,0.2,0.6,0.2,...,1.000000,0.333333,8.6,1.0,0.0,4.6,4.333333,13.666667,2.000000,0.000000
3,2010-12-26,Man United,Sunderland,H,1.267143,5.321429,11.785714,0.6,0.4,0.0,...,2.000000,0.000000,8.8,1.4,0.0,3.4,6.666667,11.666667,2.333333,0.333333
4,2010-12-26,Wolves,Wigan,A,2.078571,3.242857,3.664286,0.4,0.0,0.6,...,3.000000,0.666667,14.4,2.2,0.4,3.4,3.333333,11.000000,1.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,2023-05-28,Crystal Palace,Nott'm Forest,D,1.670000,4.013333,4.683333,0.0,0.6,0.4,...,1.666667,0.000000,9.6,1.6,0.0,3.8,5.000000,8.666667,0.666667,0.000000
3645,2023-05-28,Everton,Bournemouth,H,1.453333,4.460000,6.715000,0.0,0.4,0.6,...,3.000000,0.333333,11.6,1.8,0.2,4.6,6.333333,10.000000,1.333333,0.000000
3646,2023-05-28,Leeds,Tottenham,A,2.710000,3.738333,2.330000,0.2,0.4,0.4,...,1.666667,0.000000,9.0,1.0,0.0,5.8,4.000000,8.333333,2.666667,0.000000
3647,2023-05-28,Leicester,West Ham,H,1.918333,3.795000,3.565000,0.8,0.0,0.2,...,1.333333,0.000000,12.4,1.4,0.0,5.4,4.333333,10.000000,2.333333,0.000000


In [4]:
from sklearn.preprocessing import LabelEncoder

train_data = train_data.drop(columns=['Date'])
val_data = val_data.drop(columns=['Date'])


# Mã hóa cột HomeTeam và AwayTeam
label_encoder_home = LabelEncoder()
label_encoder_away = LabelEncoder()

train_data['HomeTeam'] = label_encoder_home.fit_transform(train_data['HomeTeam'])
train_data['AwayTeam'] = label_encoder_away.fit_transform(train_data['AwayTeam'])

val_data['HomeTeam'] = label_encoder_home.transform(val_data['HomeTeam'])
val_data['AwayTeam'] = label_encoder_away.transform(val_data['AwayTeam'])


In [5]:
label_encoder = LabelEncoder()

# Mã hóa cột 'result' thành các giá trị số
train_data['Result'] = label_encoder.fit_transform(train_data['Result'])
val_data['Result'] = label_encoder.transform(val_data['Result'])  # Chú ý: sử dụng transform cho dữ liệu validation

train_data

,HomeTeam,AwayTeam,Result,Hodds,Dodds,Aodds,home_wins,home_draws,home_losses,home_goals,...,home_oppos_yellowcards,home_oppos_redcards,away_fouls,away_yellowcards,away_redcards,away_cornerkicks,away_oppos_cornerkicks,away_oppos_fouls,away_oppos_yellowcards,away_oppos_redcards
0,1,31,0,3.042857,3.285714,2.341429,0.2,0.4,0.4,1.2,...,1.000000,0.000000,10.2,1.2,0.0,5.6,10.333333,10.000000,0.666667,0.000000
1,3,28,0,2.240000,3.214286,3.285714,0.2,0.2,0.6,1.4,...,2.333333,0.333333,9.4,0.8,0.0,5.8,7.333333,10.666667,2.000000,0.000000
2,13,34,0,1.932857,3.328571,4.092857,0.2,0.6,0.2,0.8,...,1.000000,0.333333,8.6,1.0,0.0,4.6,4.333333,13.666667,2.000000,0.000000
3,20,29,2,1.267143,5.321429,11.785714,0.6,0.4,0.0,3.0,...,2.000000,0.000000,8.8,1.4,0.0,3.4,6.666667,11.666667,2.333333,0.333333
4,36,35,0,2.078571,3.242857,3.664286,0.4,0.0,0.6,1.6,...,3.000000,0.666667,14.4,2.2,0.4,3.4,3.333333,11.000000,1.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,11,24,1,1.670000,4.013333,4.683333,0.0,0.6,0.4,0.8,...,1.666667,0.000000,9.6,1.6,0.0,3.8,5.000000,8.666667,0.666667,0.000000
3645,12,5,2,1.453333,4.460000,6.715000,0.0,0.4,0.6,1.0,...,3.000000,0.333333,11.6,1.8,0.2,4.6,6.333333,10.000000,1.333333,0.000000
3646,16,31,0,2.710000,3.738333,2.330000,0.2,0.4,0.4,0.8,...,1.666667,0.000000,9.0,1.0,0.0,5.8,4.000000,8.333333,2.666667,0.000000
3647,17,34,2,1.918333,3.795000,3.565000,0.8,0.0,0.2,1.6,...,1.333333,0.000000,12.4,1.4,0.0,5.4,4.333333,10.000000,2.333333,0.000000


In [23]:
train_data = train_data.drop(columns=['home_fouls','home_yellowcards','home_redcards','home_cornerkicks','home_oppos_cornerkicks','home_oppos_fouls',
                         'home_oppos_yellowcards','home_oppos_redcards','away_fouls','away_yellowcards','away_redcards','away_cornerkicks',
                         'away_oppos_cornerkicks','away_oppos_fouls','away_oppos_yellowcards','away_oppos_redcards'])
val_data = val_data.drop(columns=['home_fouls','home_yellowcards','home_redcards','home_cornerkicks','home_oppos_cornerkicks','home_oppos_fouls',
                         'home_oppos_yellowcards','home_oppos_redcards','away_fouls','away_yellowcards','away_redcards','away_cornerkicks',
                         'away_oppos_cornerkicks','away_oppos_fouls','away_oppos_yellowcards','away_oppos_redcards'])

In [24]:
# Prepare data
X_train = train_data.drop(columns=['Result']).values
y_train = train_data['Result'].values
X_val = val_data.drop(columns=['Result']).values
y_val = val_data['Result'].values

In [25]:
train_data

,HomeTeam,AwayTeam,Result,Hodds,Dodds,Aodds,home_wins,home_draws,home_losses,home_goals,...,away_losses,away_goals,away_oppos_goals,away_shots,away_oppos_shots,away_shotontarget,away_oppos_shotontarget,home_oppos_wins,home_oppos_draws,home_oppos_losses
0,1,31,0,3.042857,3.285714,2.341429,0.2,0.4,0.4,1.2,...,0.0,2.2,1.000000,15.2,23.333333,8.6,14.000000,0.000000,0.666667,0.333333
1,3,28,0,2.240000,3.214286,3.285714,0.2,0.2,0.6,1.4,...,0.2,1.2,1.333333,12.2,13.333333,5.2,8.666667,0.000000,0.333333,0.666667
2,13,34,0,1.932857,3.328571,4.092857,0.2,0.6,0.2,0.8,...,0.4,1.0,1.666667,12.4,13.000000,4.0,6.666667,0.333333,0.666667,0.000000
3,20,29,2,1.267143,5.321429,11.785714,0.6,0.4,0.0,3.0,...,0.2,1.6,0.666667,14.2,9.333333,6.6,4.333333,0.333333,0.666667,0.000000
4,36,35,0,2.078571,3.242857,3.664286,0.4,0.0,0.6,1.6,...,0.4,0.6,1.333333,11.0,20.666667,5.2,12.000000,0.333333,0.000000,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,11,24,1,1.670000,4.013333,4.683333,0.0,0.6,0.4,0.8,...,0.6,1.0,3.000000,12.0,14.000000,3.6,7.333333,0.000000,0.000000,1.000000
3645,12,5,2,1.453333,4.460000,6.715000,0.0,0.4,0.6,1.0,...,0.6,1.6,3.333333,13.4,14.000000,5.4,7.666667,0.000000,0.000000,1.000000
3646,16,31,0,2.710000,3.738333,2.330000,0.2,0.4,0.4,0.8,...,0.8,1.6,1.666667,9.0,10.000000,4.2,4.333333,0.333333,0.333333,0.333333
3647,17,34,2,1.918333,3.795000,3.565000,0.8,0.0,0.2,1.6,...,0.4,1.4,1.000000,7.4,11.333333,2.8,5.666667,0.666667,0.333333,0.000000


In [26]:
val_data

,HomeTeam,AwayTeam,Result,Hodds,Dodds,Aodds,home_wins,home_draws,home_losses,home_goals,...,away_losses,away_goals,away_oppos_goals,away_shots,away_oppos_shots,away_shotontarget,away_oppos_shotontarget,home_oppos_wins,home_oppos_draws,home_oppos_losses
0,12,34,0,2.140000,3.445000,3.483333,0.6,0.4,0.0,2.4,...,0.0,1.6,0.666667,18.0,8.333333,6.2,1.666667,0.666667,0.333333,0.000000
1,11,26,2,1.978333,3.383333,3.973333,0.4,0.4,0.2,0.6,...,0.4,0.8,0.666667,11.8,15.333333,3.2,3.666667,0.000000,1.000000,0.000000
2,7,36,1,2.650000,3.121667,2.835000,0.2,0.2,0.6,1.2,...,0.0,1.4,1.333333,13.6,13.666667,4.2,4.000000,0.333333,0.333333,0.333333
3,33,0,0,5.913333,4.023333,1.583333,0.0,0.2,0.8,0.6,...,0.4,1.6,2.666667,14.4,14.000000,6.0,6.666667,0.333333,0.000000,0.666667
4,22,17,0,5.266667,3.940000,1.655000,0.6,0.4,0.0,1.6,...,0.0,2.6,3.000000,15.6,13.666667,5.8,5.000000,0.333333,0.000000,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,27,31,1,3.711667,3.486667,2.038333,0.4,0.0,0.6,2.2,...,0.8,0.6,3.333333,9.4,14.000000,3.0,7.666667,0.000000,0.000000,1.000000
340,32,34,0,3.291667,3.558333,2.173333,0.4,0.0,0.6,2.2,...,0.0,3.2,2.666667,22.0,18.000000,7.6,7.666667,0.333333,0.000000,0.666667
341,17,18,2,7.793333,5.553333,1.356667,0.2,0.6,0.2,1.0,...,0.4,0.4,0.666667,15.2,13.333333,3.2,3.333333,0.333333,0.666667,0.000000
342,6,19,0,17.578333,7.785000,1.151667,0.4,0.4,0.2,1.0,...,0.4,1.4,1.000000,14.8,8.666667,4.0,1.666667,0.666667,0.000000,0.333333


# Logistic Regression

In [27]:
# Cài đặt tham số Grid Search cho LR
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],  # Hệ số điều chỉnh
    'solver': ['lbfgs', 'newton-cg', 'saga'],  # Các thuật toán tối ưu
    'class_weight': [None, 'balanced']  # Cân bằng lớp
}

grid_lr = GridSearchCV(LogisticRegression(multi_class='multinomial', max_iter=5000), 
                       param_grid=param_grid_lr, 
                       scoring='accuracy', 
                       cv=5)  # 5-fold cross-validation

grid_lr.fit(X_train, y_train)
print("Best Parameters for Logistic Regression:", grid_lr.best_params_)
#Best Parameters for Logistic Regression: {'C': 0.01, 'class_weight': None, 'solver': 'lbfgs'}

f:\Apps\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
f:\Apps\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
f:\Apps\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
f:\Apps\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be remove

Best Parameters for Logistic Regression: {'C': 0.01, 'class_weight': None, 'solver': 'newton-cg'}


f:\Apps\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [28]:
# Logistic Regression
best_lr = grid_lr.best_estimator_
y_pred_lr = best_lr.predict(X_val)
print("Accuracy for Logistic Regression:", accuracy_score(y_val, y_pred_lr))
print(classification_report(y_val, y_pred_lr))

Accuracy for Logistic Regression: 0.5261627906976745
              precision    recall  f1-score   support

           0       0.55      0.52      0.53       126
           1       0.00      0.00      0.00        78
           2       0.52      0.82      0.63       140

    accuracy                           0.53       344
   macro avg       0.35      0.45      0.39       344
weighted avg       0.41      0.53      0.45       344



f:\Apps\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Apps\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Apps\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Random Forest

In [29]:
# Cài đặt tham số Grid Search cho RF
param_grid_rf = {
    'n_estimators': [100, 200, 300],  # Số lượng cây
    'max_depth': [None, 10, 20, 30],  # Độ sâu tối đa của cây
    'min_samples_split': [2, 5, 10],  # Số mẫu tối thiểu để chia nhánh
    'class_weight': [None, 'balanced']  # Cân bằng lớp
}

grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), 
                       param_grid=param_grid_rf, 
                       scoring='accuracy', 
                       cv=5)

grid_rf.fit(X_train, y_train)
print("Best Parameters for Random Forest:", grid_rf.best_params_)
#Best Parameters for Random Forest: {'class_weight': None, 'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 300}

Best Parameters for Random Forest: {'class_weight': None, 'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 200}


In [30]:
# Random Forest
best_rf = grid_rf.best_estimator_
y_pred_rf = best_rf.predict(X_val)
print("Accuracy for Random Forest:", accuracy_score(y_val, y_pred_rf))
print(classification_report(y_val, y_pred_rf))

Accuracy for Random Forest: 0.7674418604651163
              precision    recall  f1-score   support

           0       0.78      0.78      0.78       126
           1       1.00      0.51      0.68        78
           2       0.70      0.90      0.79       140

    accuracy                           0.77       344
   macro avg       0.83      0.73      0.75       344
weighted avg       0.80      0.77      0.76       344



# XGBoost

In [22]:
# Cài đặt tham số Grid Search cho XGB
param_grid_xgb = {
    'n_estimators': [100, 200, 300],  # Số lượng cây
    'max_depth': [3, 5, 7],  # Độ sâu tối đa của cây
    'learning_rate': [0.01, 0.1, 0.2],  # Tốc độ học
    'subsample': [0.8, 1.0],  # Tỷ lệ mẫu ngẫu nhiên
    'colsample_bytree': [0.8, 1.0]  # Tỷ lệ cột được chọn cho mỗi cây
}

grid_xgb = GridSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'), 
                        param_grid=param_grid_xgb, 
                        scoring='accuracy', 
                        cv=5)

grid_xgb.fit(X_train, y_train)
print("Best Parameters for XGBoost:", grid_xgb.best_params_)


NameError: name 'XGBClassifier' is not defined

In [ ]:
# XGBoost
best_xgb = grid_xgb.best_estimator_
y_pred_xgb = best_xgb.predict(X_val)
print("Accuracy for XGBoost:", accuracy_score(y_val, y_pred_xgb))
print(classification_report(y_val, y_pred_xgb))